# Playing with ansible API

- https://docs.ansible.com/ansible/dev_guide/developing_api.html

- https://www.ansible.com/blog/how-to-extend-ansible-through-plugins — excellent overview of ansible extension points

- <img alt="Extending Ansible cover" src="Extending_Ansible_cover.jpg" style="width: 20%; float: right">
  [*Extending Ansible* book][3] by Rishabh Das (2016, I think describes pre-2.0?)  
  Free sample including API chapter at https://www.ansible.com/extending-ansible

- [`lib/ansible/adhoc.py`][1] and [`lib/ansible/playbook.py`][2] are simple usage examples.

[1]: https://github.com/ansible/ansible/blob/devel/lib/ansible/cli/adhoc.py
[2]: https://github.com/ansible/ansible/blob/devel/lib/ansible/cli/playbook.py
[3]: https://www.packtpub.com/networking-and-servers/extending-ansible

In [1]:
from ansible import constants as C
from ansible.cli import CLI
from ansible.errors import AnsibleError, AnsibleOptionsError, AnsibleParserError
from ansible.executor.task_queue_manager import TaskQueueManager
from ansible.inventory import Inventory
#from ansible.module_utils._text import to_text
from ansible.parsing.dataloader import DataLoader
#from ansible.parsing.splitter import parse_kv
from ansible.playbook.play import Play
#from ansible.plugins import get_all_plugin_loaders
#from ansible.utils.vars import load_extra_vars
#from ansible.utils.vars import load_options_vars
from ansible.vars import VariableManager


In [2]:
variable_manager = VariableManager()

In [3]:
loader = DataLoader()

In [4]:
inventory = Inventory(loader=loader, variable_manager=variable_manager)

In [5]:
passwords = {}

An `options` object is needed, many places in code require specific attributes to exist.
Could build one but easier to use the CLI arguments parser to provide them.

In [6]:
#import argparse
#options = argparse.Namespace(module_path=None, forks=C.DEFAULT_FORKS, become=C.DEFAULT_BECOME)
parser = CLI.base_parser(module_opts=True, fork_opts=True, runas_opts=True, check_opts=True)
options, extra_args = parser.parse_args([])
options

<Values at 0x7efc74060320: {'sudo_user': None, 'ask_su_pass': False, 'diff': False, 'verbosity': 0, 'become_ask_pass': False, 'su': False, 'forks': 5, 'syntax': None, 'su_user': None, 'become_method': 'sudo', 'check': False, 'become_user': None, 'become': False, 'sudo': False, 'module_path': None, 'ask_sudo_pass': False}>

In [7]:
def task_queue_manager():
    return TaskQueueManager(
        inventory=inventory,
        variable_manager=variable_manager,
        loader=loader,
        options=options,
        passwords=passwords,
        #stdout_callback=cb,
        #run_additional_callbacks=C.DEFAULT_LOAD_CALLBACK_PLUGINS,
        #run_tree=run_tree,
    )
tqm = task_queue_manager()

## Getting a Play data structure

In [8]:
import yaml

In [9]:
play1 = Play.load(yaml.load('''
hosts: localhost
tasks:
  - command: zenity --question --text="WORKS! Proceed?"
'''))
play1.tasks

[BLOCK(uuid=0242556f-fc21-9054-8f0b-000000000002)(id=139622390049032)(parent=None)]

In [10]:
tqm.run(play1)


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [command] *****************************************************************
changed: [localhost]


0

## TaskQueueManager is stateful
Look what happens after a play fails:

In [11]:
failing_play = Play.load(yaml.load('''
hosts: localhost
tasks:
  - command: 'false'
'''))
tqm.run(failing_play)


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [command] *****************************************************************
fatal: [localhost]: FAILED! => {"changed": true, "cmd": ["false"], "delta": "0:00:00.002526", "end": "2017-06-12 14:29:49.825338", "failed": true, "rc": 1, "start": "2017-06-12 14:29:49.822812", "stderr": "", "stderr_lines": [], "stdout": "", "stdout_lines": []}


2

In [12]:
tqm.run(play1)


PLAY [localhost] ***************************************************************


2

 => tqm will not run anything more, it will just return error exit code :-(

In [13]:
TaskQueueManager.RUN_FAILED_HOSTS

2

### Two solutions
1. Create new TaskQueueManager every time.
2. `tqm.clear_failed_hosts()`.

### Make it easier to run

In [14]:
def run(code):
    play = Play.load(yaml.safe_load(code))
    task_queue_manager().run(play)

In [15]:
run("""
tasks:
  - command: echo foo
""")

AnsibleParserError: the field 'hosts' is required but was not set

Oops.  That was not convenient enough.  Also, what a huge stacktrace :-(

In [ ]:
import sys
import traceback
def run(code):
    try:
        play = Play.load(yaml.load(code))
        task_queue_manager().run(play)
    except (yaml.YAMLError, AnsibleParserError) as e:
        # Printing errors will look different in a Jupyter kernel, but for now stderr is fine.
        print(''.join(traceback.format_exception_only(type(e), e)), file=sys.stderr)

In [ ]:
run("""
tasks
- syntax error: missing semicolon above after `tasks`
""")

In [ ]:
run("""
tasks:
- command: echo foo
""")

Okay, much better errors!  Back to making it easy to write simple plays:

In [ ]:
def play_from_code(code):
    """Support one task, list of tasks, or whole play without hosts."""
    data = orig_data = yaml.safe_load(code)
    if isinstance(data, dict) and 'tasks' not in data:
        data = [data]
    if isinstance(data, list):
        data = dict(tasks=data)
    if not isinstance(data, dict):
        raise AnsibleParserError("Expected task, list of tasks, or play, got {}".format(type(orig_data)))
    if 'hosts' not in data:
        data['hosts'] = 'localhost'
    return Play.load(data)

def run(code):
    try:
        task_queue_manager().run(play_from_code(code))
    except (yaml.YAMLError, AnsibleParserError) as e:
        # Printing errors will look different in a Jupyter kernel, but for now stderr is fine.
        print(''.join(traceback.format_exception_only(type(e), e)), file=sys.stderr)

In [ ]:
run("""command: echo foo""")

### Have we solved stateful TQM the right way?
We're using a fresh `TaskQueueManager` every time, so do we still carry *any* state from cell to cell?
Can we set a variable and later use it?

In [ ]:
run("""set_fact: var=1""")

In [ ]:
run("""debug: msg={{var}}""")

# TODO UNSOLVED
Wait, what's that "Gathering Facts" from localhost every time?

In [ ]:
def run(code):
    try:
        tqm.clear_failed_hosts()
        tqm.run(play_from_code(code))
    except (yaml.YAMLError, AnsibleParserError) as e:
        # Printing errors will look different in a Jupyter kernel, but for now stderr is fine.
        print(e, file=sys.stderr)


In [ ]:
run("""debug: msg={{var}}""")

## How is a play processed before execution?
### Let's consider a more complex play.

In [ ]:
play = to_play('''
vars:
  play_local_variable: 'abc'
  
tasks:

- debug: 'msg=The var equals {{play_local_variable}}'

- command: pwd
  register: pwd_out  # Sets a variable

- set_fact: global1=value1 global2=value2  # Some more vars

- with_items: [play_local_variable, pwd_out, global1, global2]  # A loop!
  debug: var={{item}}
''')
task_queue_manager().run(play)

*__Tip__: Ansible objects have `.serialize()`, handy for exploring.*

In [ ]:
play.serialize()

In [ ]:
play.get_vars()

In [ ]:
variable_manager.get_vars(loader=loader, play=play)

In [ ]:
play.compile()

^^ These are the pre_tasks, roles, tasks, and post_tasks

In [ ]:
[list(block.block) for block in play.compile()]


In [ ]:
play.compile()[1].serialize()

## Variables

In [ ]:
from ansible.vars import preprocess_vars
preprocess_vars(yaml.load('''
hosts: localhost
tasks:
  - command: 'false'
'''))

# Peek under the hood of IPyKernel running *this* notebook?

In [ ]:
import sys
sorted(sys.modules)

In [ ]:
import ipykernel.ipkernel

In [ ]:
import gc
[obj for obj in gc.get_referrers(ipykernel.ipkernel.IPythonKernel) 
 if isintance(obj,ipykernel.ipkernel.IPythonKernel)]